In [8]:
import numpy as np
import os
import csv
from collections import defaultdict
import MDAnalysis as mda
from MDAnalysis.lib.distances import distance_array
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)

# ==== CONFIG ====
topology = "oligomer.pdb"
trajectory = "short_trajectory.dcd"
output_dir = "results"
os.makedirs(output_dir, exist_ok=True)
log_path = os.path.join(output_dir, "log.txt")

u = mda.Universe(topology, trajectory)
all_atoms = u.select_atoms("protein")
select_atoms = u.select_atoms("name BB")

# ==== Compute system Rg and max_radius ====
rg_values = []
for ts in u.trajectory:
    coords = all_atoms.positions
    com = coords.mean(axis=0)
    rg_frame = np.sqrt(np.sum((coords - com) ** 2) / len(coords))
    rg_values.append(rg_frame)

rg_avg = np.mean(rg_values)
################################
percent_cover=0.5
max_radius = rg_avg * percent_cover
#################################
print("Radius of gyration:", rg_avg)
print("Maximum radii sampled", max_radius)

# ==== PARAMS ====
num_directions = int(500 * (rg_avg / 40)**2)
step_size = 0.25
contact_cutoff = 1
frame_interval = 1
directions = np.array([
    [np.cos(i * np.pi * (3. - np.sqrt(5.))) * np.sqrt(1 - (1 - (i / (num_directions - 1)) * 2) ** 2),
     1 - (i / (num_directions - 1)) * 2,
     np.sin(i * np.pi * (3. - np.sqrt(5.))) * np.sqrt(1 - (1 - (i / (num_directions - 1)) * 2) ** 2)]
    for i in range(num_directions)
])

# ==== STORAGE ====
rg_per_frame = []
direct_map = defaultdict(set)
indirect_map = defaultdict(set)

log = open(log_path, "w")

for i, ts in tqdm(enumerate(u.trajectory[::frame_interval], start=1), desc="Processing frames"):
    frame_id = f"{i:04d}"
    all_atoms = u.select_atoms("protein")
    coords = all_atoms.positions
    com = coords.mean(axis=0)
    log.write(f"Frame {frame_id}...\n")

    void_hit_atoms = []
    for direction in directions:
        for r in np.arange(step_size, max_radius, step_size):
            point = com + r * direction
            dists = distance_array(point.reshape(1, 3), select_atoms.positions)[0]
            contacts = np.where(dists < contact_cutoff)[0]
            if len(contacts):
                for idx in contacts:
                    void_hit_atoms.append(select_atoms[idx])
                break

    if not void_hit_atoms:
        continue

    void_hit_coords = np.array([atom.position for atom in void_hit_atoms])

    # ==== DIRECT ====
    direct_keys = set()
    for atom in void_hit_atoms:
        key = (atom.resid, atom.segid.strip())
        direct_map[key].add(i)
        direct_keys.add(key)


    # ==== PDB WRITE ====
    direct_residues = [res for res in all_atoms.residues if (res.resid, res.segid.strip()) in direct_keys]


    if direct_residues:
        direct_atoms = mda.Merge(*[res.atoms for res in direct_residues])
        direct_atoms.atoms.write(f"{output_dir}/void_direct_{frame_id}.pdb")

    # ==== Rg of void-contact BB atoms ====
    coords = void_hit_coords
    com = coords.mean(axis=0)
    rg_frame = np.sqrt(np.sum((coords - com) ** 2) / len(coords))
    rg_per_frame.append((frame_id, rg_frame))

log.close()

# ==== ENFORCE MUTUAL EXCLUSIVITY ====
direct_keys_all = set(direct_map.keys())

# ==== WRITE CSVs ====
with open(f"{output_dir}/rg_per_frame.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["frame", "Rg"])
    writer.writerows(rg_per_frame)

for fname, datamap in [
    ("direct_contact_residues.csv", direct_map)
]:
    with open(os.path.join(output_dir, fname), "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["resid", "segid", "frames_present", "count"])
        for (resid, segid), frames in sorted(datamap.items(), key=lambda x: -len(x[1])):
            writer.writerow([resid, segid, sorted(frames), len(frames)])

print("✅ Saved direct, indirect, combined contact PDBs and CSVs")
print("\n📊 Spread of Void Atom Displacements:")
rg_array = np.array([rg for _, rg in rg_per_frame])
print(f"\n📈 Rg stats — min: {np.min(rg_array):.2f}, max: {np.max(rg_array):.2f}, mean: {np.mean(rg_array):.2f}")


Radius of gyration: 41.285275
Maximum radii sampled 20.642637


Processing frames: 20it [00:22,  1.13s/it]

✅ Saved direct, indirect, combined contact PDBs and CSVs

📊 Spread of Void Atom Displacements:

📈 Rg stats — min: 14.32, max: 15.79, mean: 15.20
